# Simulating Vehicles Passing Through a 4-way Stop Sign

Final Project by Ethan Haley
for Dr. Fulton's CUNY SPS DATA 604 Summer '22

## Part 1 -- Simulating a flow into a 4-way stop sign intersection

The idea here is to create and visualize the flow of stochastically generated vehicles into and through a common intersection.  First and foremost, my project is built upon Bilal Himite's traffic visualization code repository here 

https://github.com/BilHim/trafficSimulator

which that author presents in a blog post here

https://towardsdatascience.com/simulating-traffic-flow-in-python-ee1eab4dd20f

A large component of Professor Larry Fulton's 604 course is the verification of simulations, and so I approached this project from the standpoint of using visualizations to verify what was happening and why. "I'll believe it when I see it", in a nutshell.  We may not know what some simulations should look like, but most of us have sat at enough stop signs to have had the time to analyze them thoroughly and know what the traffic flow looks like. 

The previously mentioned repo that I used as the basis for my project was written by an engineer who understands the physics of motion very well, and who provides a very nice scaffolding for visualization of motion.  It's based on the  "intelligent driver model" described in 2000 by Treiber, Hennecke, and Helbing here https://journals.aps.org/pre/abstract/10.1103/PhysRevE.62.1805 . 

The idea is to model every vehicle as an individual Agent who has their own driving preferences but is often constrained by the car in front of them.

For example, here is a snapshot in time of one of my simulated intersections:

<img src='trafficSimulator/SIM_1_42s.png' width="600" height="600">

The yellow car facing North on the South side of the intersection is waiting in a queue with the other yellow car.  The green car heading West through the intersection has just been released from the queue and now has no one in front of it, so it is accelerating at its desired rate.  When it clears the way in a second, the two yellow cars will have to figure out who gets to join the Westward route first behind the green car, since they both have their signals on indicating that they will be turning that same direction.  

The author of the code that I based this on used timed stoplights that released everyone on a road at once, for a set amount of time, and allowed them to collide freely, since that was not the topic of interest in his project.  So my task was to figure out the logic of managing the vehicles in a situation that turns out to be very different from a stoplight, surprisingly.  

### Why simulate this Agent-Agent interaction?

From an immediate standpoint, the goal was to simulate how cars entered the intersection area and how that affected the flow through the crossroads and onward.  A random Poisson distribution that might model the timing of cars entering the road off-screen becomes a very different distribution a couple of blocks later, when different drivers have interacted and are preparing to stop at the 4-way.  I wanted to see how different driving styles affected the overall traffic movement.  This of course proved to be too complex for me!  But I was at least able to simulate a new semi-random flow into the intersection under a variety of parameters, and the point is to use those distributions as input to a SimPy model that will control the movement through the intersection in an analyzable manner.

Here is the "intelligent driver model" used to explain how independent driver Agents act in a group:

<img src='trafficSimulator/TrafficFlowChart.png' width="600" height="600">

In [1]:
import pandas as pd
import numpy as np
import simpy as sp
from trafficSimulator import *

pygame 2.1.2 (SDL 2.0.18, Python 3.7.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
sim = Simulation()
np.random.seed(604)
# The road layout is shaped with these
n = 12   # how many straight segments to a turn (higher is more circular/smoother)
a = 2    # affects the width of the roads
b = 8   # adds curvature to the corners
l = 180  # makes the window a smaller square (smaller = shorter roads). 
        ### I'm making it high to bunch together incoming cars more, before we see them, in a stab at reality

# Nodes   ----> "right" and "left" orient to the way the compass point is driving towards the center
WEST_RIGHT_START = (-b-l, a)
WEST_LEFT_START = (-b-l, -a)

SOUTH_RIGHT_START = (a, b+l)
SOUTH_LEFT_START = (-a, b+l)

EAST_RIGHT_START = (b+l, -a)
EAST_LEFT_START = (b+l, a)

NORTH_RIGHT_START = (-a, -b-l)
NORTH_LEFT_START = (a, -b-l)


WEST_RIGHT = (-b, a)
WEST_LEFT =	(-b, -a)

SOUTH_RIGHT = (a, b)
SOUTH_LEFT = (-a, b)

EAST_RIGHT = (b, -a)
EAST_LEFT = (b, a)

NORTH_RIGHT = (-a, -b)
NORTH_LEFT = (a, -b)

# Roads
WEST_INBOUND = (WEST_RIGHT_START, WEST_RIGHT)
SOUTH_INBOUND = (SOUTH_RIGHT_START, SOUTH_RIGHT)
EAST_INBOUND = (EAST_RIGHT_START, EAST_RIGHT)
NORTH_INBOUND = (NORTH_RIGHT_START, NORTH_RIGHT)

WEST_OUTBOUND = (WEST_LEFT, WEST_LEFT_START)
SOUTH_OUTBOUND = (SOUTH_LEFT, SOUTH_LEFT_START)
EAST_OUTBOUND = (EAST_LEFT, EAST_LEFT_START)
NORTH_OUTBOUND = (NORTH_LEFT, NORTH_LEFT_START)

WEST_STRAIGHT = (WEST_RIGHT, EAST_LEFT)
SOUTH_STRAIGHT = (SOUTH_RIGHT, NORTH_LEFT)
EAST_STRAIGHT = (EAST_RIGHT, WEST_LEFT)
NORTH_STRAIGHT = (NORTH_RIGHT, SOUTH_LEFT)

WEST_RIGHT_TURN = turn_road(WEST_RIGHT, SOUTH_LEFT, TURN_RIGHT, n)
WEST_LEFT_TURN = turn_road(WEST_RIGHT, NORTH_LEFT, TURN_LEFT, n)

SOUTH_RIGHT_TURN = turn_road(SOUTH_RIGHT, EAST_LEFT, TURN_RIGHT, n)
SOUTH_LEFT_TURN = turn_road(SOUTH_RIGHT, WEST_LEFT, TURN_LEFT, n)

EAST_RIGHT_TURN = turn_road(EAST_RIGHT, NORTH_LEFT, TURN_RIGHT, n)
EAST_LEFT_TURN = turn_road(EAST_RIGHT, SOUTH_LEFT, TURN_LEFT, n)

NORTH_RIGHT_TURN = turn_road(NORTH_RIGHT, WEST_LEFT, TURN_RIGHT, n)
NORTH_LEFT_TURN = turn_road(NORTH_RIGHT, EAST_LEFT, TURN_LEFT, n)

sim.create_roads([
    WEST_INBOUND,
    SOUTH_INBOUND,
    EAST_INBOUND,
    NORTH_INBOUND,

    WEST_OUTBOUND,
    SOUTH_OUTBOUND,
    EAST_OUTBOUND,
    NORTH_OUTBOUND,

    WEST_STRAIGHT,
    SOUTH_STRAIGHT,
    EAST_STRAIGHT,
    NORTH_STRAIGHT,

    *WEST_RIGHT_TURN,  # so this is n little roads
    *WEST_LEFT_TURN,   # and this is too

    *SOUTH_RIGHT_TURN,
    *SOUTH_LEFT_TURN,

    *EAST_RIGHT_TURN,
    *EAST_LEFT_TURN,

    *NORTH_RIGHT_TURN,
    *NORTH_LEFT_TURN
])

def road(a):
    # build many (n, from above, to be precise) small roads to model a turn
    return range(a, a+n)    

sim.create_gen({
'vehicle_rate': 40,
'vehicles':[
    [3, {'path': [0, 8, 6], 'direction': 'STRAIGHT'}],   # [WEST_INBOUND, WEST_STRAIGHT, EAST_OUTBOUND],  from above roads array
    [1, {'path': [0, *road(12), 5], 'direction': 'RIGHT'}],   # [WEST_INBOUND, *WEST_RIGHT_TURN, SOUTH_OUTBOUND],  from above roads array
    [1, {'path': [0, *road(12+n), 7], 'direction': 'LEFT'}],   # [WEST_INBOUND, *WEST_LEFT_TURN, NORTH_OUTBOUND],  from above roads array

    [3, {'path': [1, 9, 7], 'direction': 'STRAIGHT'}],   # [SOUTH_INBOUND, SOUTH_STRAIGHT, NORTH_OUTBOUND]
    [1, {'path': [1, *road(12+2*n), 6], 'direction': 'RIGHT'}],   # etc
    [1, {'path': [1, *road(12+3*n), 4], 'direction': 'LEFT'}],


    [3, {'path': [2, 10, 4], 'direction': 'STRAIGHT'}],  # [EAST_INBOUND, EAST_STRAIGHT, EAST_OUTBOUND]
    [1, {'path': [2, *road(12+4*n), 7], 'direction': 'RIGHT'}],  # etc
    [1, {'path': [2, *road(12+5*n), 5], 'direction': 'LEFT'}],

    [3, {'path': [3, 11, 5], 'direction': 'STRAIGHT'}],  # [NORTH_INBOUND, etc]
    [1, {'path': [3, *road(12+6*n), 4], 'direction': 'RIGHT'}],
    [1, {'path': [3, *road(12+7*n), 6], 'direction': 'LEFT'}]
]})

sim.create_signal([[0],[1],[2],[3]]) ;  # using 4 separate groups here so that each stop sign can function individually
                                        # Though my signals are permanently on "red light" for my project

Note that if you run the cell below, it should open a pygame window that shows the simulation, which may or may not be what you want.  But if so, you can stop it by coming back to this notebook and hitting the black square under the word "Kernel" in the toolbar at the top.

In [3]:
# Start simulation

win = Window(sim)
win.zoom = 5  # use two fingers to change this zoom while running
win.run(steps_per_update=3)  # controls the speed of the visualization (higher = faster)

KeyboardInterrupt: 

The first traffic flow is generated by these parameters, and is run long enough to let 250 cars through the intersection.   The only parameters I'll change for the next generator simulations are the `veh_rate`, which controls how frequently vehicles are stochastically added to the roads, and the last 4 tuple parameters, which control the ultimate directions of each of the cars that is generated on each of the 4 roads.  In SIM_1, all 4 roads have the same directional weightings: (3, 1, 1).  These numbers are the weightings for a uniform sampling.  So picture a 5-sided die being rolled 250 times here, and 3 of the 5 faces of the die say "STRAIGHT", while one each say "LEFT" and "RIGHT".  

While the visible screen defaults to only showing a fraction of the `lenroad`=180 units, you can zoom out with two fingers to see the generated vehicles from their inception.  The idea was to see how much cars bunch up before they get to the visible intersection, in the cases where the drivers had different acceleration, velocity, and braking tendencies.  The `qzone` parameter determines how far back from the intersection the car is considered "stopped" or "in line".  I have it set to 1.5 carlengths in these simulations.  The other params determine how the individual driver drives and where people slow down and stop on all 4 roads.  I would like to experiment with those more, but there is kind of a delicate balance to the "logic" of keeping the traffic flowing without causing constant crashes in a non-threadsafe environment like pygame.  Which is why every intersection needs something like SimPy, which can keep a strict, single-thread order to every time step, unlike pygame, and unlike people in the real world.  Although it really is surprising we humans can handle something as complex as a 4-way stop sign, to be fair to us.  

In [ ]:
SIM_1 = {'npseed': 604, 'veh_rate': 40, 'lenroad': 180, 'v': 15, 'a': 4, 'b': 4,
        "LEFT": 3, "STRAIGHT": 2, "RIGHT": 2, 'qzone': 6, 'slow_dist': 22,
        'slow_fac': .4, 'stop_dist': 7, 'road_0': (3,1,1), 'road_1': (3,1,1), 
         'road_2': (3,1,1), 'road_3': (3,1,1)}

In [2]:
def logDF(simlog):
    return pd.DataFrame(simlog, columns=['Road', 'enter_Q', 
                                         'time_needed', 'exit_Q', 'direction'])

In [ ]:
SIM_1_DF = logDF(sim.popped)
SIM_1_DF.head()

In [ ]:
#SIM_1_DF.to_csv("SIM_1_DF.csv")

For the second flow generator, I'll increase the generation rate from 40 to 60, and I'll make most cars want to head East.  This will jam up the traffic, mostly from the increased generation rate.

In [ ]:
SIM_2 = {'npseed': 604, 'veh_rate': 60, 'lenroad': 180, 'v': 15, 'a': 4, 'b': 4,
        "LEFT": 3, "STRAIGHT": 2, "RIGHT": 2, 'qzone': 6, 'slow_dist': 22,
        'slow_fac': .4, 'stop_dist': 7, 'road_0': (4,1,1), 'road_1': (2,1,2), 
         'road_2': (1,1,1), 'road_3': (2,1,4)}

In [8]:
sim2 = Simulation()
np.random.seed(604)
# The road layout is shaped with these
n = 12   # how many straight segments to a turn (higher is more circular/smoother)
a = 2    # affects the width of the roads
b = 8   # adds curvature to the corners
l = 180  # makes the window a smaller square (smaller = shorter roads). 
        ### I'm making it high to bunch together incoming cars more, before we see them, in a stab at reality

# Nodes   ----> "right" and "left" orient to the way the compass point is driving towards the center
WEST_RIGHT_START = (-b-l, a)
WEST_LEFT_START = (-b-l, -a)

SOUTH_RIGHT_START = (a, b+l)
SOUTH_LEFT_START = (-a, b+l)

EAST_RIGHT_START = (b+l, -a)
EAST_LEFT_START = (b+l, a)

NORTH_RIGHT_START = (-a, -b-l)
NORTH_LEFT_START = (a, -b-l)


WEST_RIGHT = (-b, a)
WEST_LEFT =	(-b, -a)

SOUTH_RIGHT = (a, b)
SOUTH_LEFT = (-a, b)

EAST_RIGHT = (b, -a)
EAST_LEFT = (b, a)

NORTH_RIGHT = (-a, -b)
NORTH_LEFT = (a, -b)

# Roads
WEST_INBOUND = (WEST_RIGHT_START, WEST_RIGHT)
SOUTH_INBOUND = (SOUTH_RIGHT_START, SOUTH_RIGHT)
EAST_INBOUND = (EAST_RIGHT_START, EAST_RIGHT)
NORTH_INBOUND = (NORTH_RIGHT_START, NORTH_RIGHT)

WEST_OUTBOUND = (WEST_LEFT, WEST_LEFT_START)
SOUTH_OUTBOUND = (SOUTH_LEFT, SOUTH_LEFT_START)
EAST_OUTBOUND = (EAST_LEFT, EAST_LEFT_START)
NORTH_OUTBOUND = (NORTH_LEFT, NORTH_LEFT_START)

WEST_STRAIGHT = (WEST_RIGHT, EAST_LEFT)
SOUTH_STRAIGHT = (SOUTH_RIGHT, NORTH_LEFT)
EAST_STRAIGHT = (EAST_RIGHT, WEST_LEFT)
NORTH_STRAIGHT = (NORTH_RIGHT, SOUTH_LEFT)

WEST_RIGHT_TURN = turn_road(WEST_RIGHT, SOUTH_LEFT, TURN_RIGHT, n)
WEST_LEFT_TURN = turn_road(WEST_RIGHT, NORTH_LEFT, TURN_LEFT, n)

SOUTH_RIGHT_TURN = turn_road(SOUTH_RIGHT, EAST_LEFT, TURN_RIGHT, n)
SOUTH_LEFT_TURN = turn_road(SOUTH_RIGHT, WEST_LEFT, TURN_LEFT, n)

EAST_RIGHT_TURN = turn_road(EAST_RIGHT, NORTH_LEFT, TURN_RIGHT, n)
EAST_LEFT_TURN = turn_road(EAST_RIGHT, SOUTH_LEFT, TURN_LEFT, n)

NORTH_RIGHT_TURN = turn_road(NORTH_RIGHT, WEST_LEFT, TURN_RIGHT, n)
NORTH_LEFT_TURN = turn_road(NORTH_RIGHT, EAST_LEFT, TURN_LEFT, n)

sim2.create_roads([
    WEST_INBOUND,
    SOUTH_INBOUND,
    EAST_INBOUND,
    NORTH_INBOUND,

    WEST_OUTBOUND,
    SOUTH_OUTBOUND,
    EAST_OUTBOUND,
    NORTH_OUTBOUND,

    WEST_STRAIGHT,
    SOUTH_STRAIGHT,
    EAST_STRAIGHT,
    NORTH_STRAIGHT,

    *WEST_RIGHT_TURN,  # so this is n little roads
    *WEST_LEFT_TURN,   # and this is too

    *SOUTH_RIGHT_TURN,
    *SOUTH_LEFT_TURN,

    *EAST_RIGHT_TURN,
    *EAST_LEFT_TURN,

    *NORTH_RIGHT_TURN,
    *NORTH_LEFT_TURN
])

def road(a):
    # build many (n, from above, to be precise) small roads to model a turn
    return range(a, a+n)    

sim2.create_gen({
'vehicle_rate': 60,   # changed this from previous sim
'vehicles':[
    [4, {'path': [0, 8, 6], 'direction': 'STRAIGHT'}],   # [WEST_INBOUND, WEST_STRAIGHT, EAST_OUTBOUND],  from above roads array
    [1, {'path': [0, *road(12), 5], 'direction': 'RIGHT'}],   # [WEST_INBOUND, *WEST_RIGHT_TURN, SOUTH_OUTBOUND],  from above roads array
    [1, {'path': [0, *road(12+n), 7], 'direction': 'LEFT'}],   # [WEST_INBOUND, *WEST_LEFT_TURN, NORTH_OUTBOUND],  from above roads array

    [2, {'path': [1, 9, 7], 'direction': 'STRAIGHT'}],   # [SOUTH_INBOUND, SOUTH_STRAIGHT, NORTH_OUTBOUND]
    [1, {'path': [1, *road(12+2*n), 6], 'direction': 'RIGHT'}],   # etc
    [2, {'path': [1, *road(12+3*n), 4], 'direction': 'LEFT'}],


    [1, {'path': [2, 10, 4], 'direction': 'STRAIGHT'}],  # [EAST_INBOUND, EAST_STRAIGHT, EAST_OUTBOUND]
    [1, {'path': [2, *road(12+4*n), 7], 'direction': 'RIGHT'}],  # etc
    [1, {'path': [2, *road(12+5*n), 5], 'direction': 'LEFT'}],

    [1, {'path': [3, 11, 5], 'direction': 'STRAIGHT'}],  # [NORTH_INBOUND, etc]
    [2, {'path': [3, *road(12+6*n), 4], 'direction': 'RIGHT'}],
    [4, {'path': [3, *road(12+7*n), 6], 'direction': 'LEFT'}]
]})

sim2.create_signal([[0],[1],[2],[3]]) ;  # using 4 separate groups here so that each stop sign can function individually
                                        # Though my signals are permanently on "red light" for my project

Note that if you run the cell below, it should open a pygame window that shows the simulation, which may or may not be what you want.  But if so, you can stop it by coming back to this notebook and hitting the black square under the word "Kernel" in the toolbar at the top.

In [9]:
# Start simulation

win = Window(sim2)
win.zoom = 5  # use two fingers to change this zoom while running
win.run(steps_per_update=3)  # controls the speed of the visualization (higher = faster)

KeyboardInterrupt: 

In [11]:
SIM_2_DF = logDF(sim2.popped)
SIM_2_DF.head()

,Road,enter_Q,time_needed,exit_Q,direction
0,2,13.98,2,14.95,RIGHT
1,1,14.98,2,16.91,RIGHT
2,1,18.22,2,19.47,STRAIGHT
3,0,19.02,3,20.95,LEFT
4,1,20.80,2,22.73,STRAIGHT


In [12]:
len(sim2.popped)

152

That was run for longer than the first one, and had more cars entering the gridlock.  But it processed far fewer, because the lanes jammed up.  

In [13]:
#SIM_2_DF.to_csv("SIM_2_DF.csv")

For the last flow generator, I'll dial it back to the 40 rate, but keep the directional bent towards the East.

In [14]:
SIM_3 = {'npseed': 604, 'veh_rate': 40, 'lenroad': 180, 'v': 15, 'a': 4, 'b': 4,
        "LEFT": 3, "STRAIGHT": 2, "RIGHT": 2, 'qzone': 6, 'slow_dist': 22,
        'slow_fac': .4, 'stop_dist': 7, 'road_0': (4,1,1), 'road_1': (2,1,2), 
         'road_2': (1,1,1), 'road_3': (2,1,4)}

In [4]:
sim3 = Simulation()
np.random.seed(604)
# The road layout is shaped with these
n = 12   # how many straight segments to a turn (higher is more circular/smoother)
a = 2    # affects the width of the roads
b = 8   # adds curvature to the corners
l = 180  # makes the window a smaller square (smaller = shorter roads). 
        ### I'm making it high to bunch together incoming cars more, before we see them, in a stab at reality

# Nodes   ----> "right" and "left" orient to the way the compass point is driving towards the center
WEST_RIGHT_START = (-b-l, a)
WEST_LEFT_START = (-b-l, -a)

SOUTH_RIGHT_START = (a, b+l)
SOUTH_LEFT_START = (-a, b+l)

EAST_RIGHT_START = (b+l, -a)
EAST_LEFT_START = (b+l, a)

NORTH_RIGHT_START = (-a, -b-l)
NORTH_LEFT_START = (a, -b-l)


WEST_RIGHT = (-b, a)
WEST_LEFT =	(-b, -a)

SOUTH_RIGHT = (a, b)
SOUTH_LEFT = (-a, b)

EAST_RIGHT = (b, -a)
EAST_LEFT = (b, a)

NORTH_RIGHT = (-a, -b)
NORTH_LEFT = (a, -b)

# Roads
WEST_INBOUND = (WEST_RIGHT_START, WEST_RIGHT)
SOUTH_INBOUND = (SOUTH_RIGHT_START, SOUTH_RIGHT)
EAST_INBOUND = (EAST_RIGHT_START, EAST_RIGHT)
NORTH_INBOUND = (NORTH_RIGHT_START, NORTH_RIGHT)

WEST_OUTBOUND = (WEST_LEFT, WEST_LEFT_START)
SOUTH_OUTBOUND = (SOUTH_LEFT, SOUTH_LEFT_START)
EAST_OUTBOUND = (EAST_LEFT, EAST_LEFT_START)
NORTH_OUTBOUND = (NORTH_LEFT, NORTH_LEFT_START)

WEST_STRAIGHT = (WEST_RIGHT, EAST_LEFT)
SOUTH_STRAIGHT = (SOUTH_RIGHT, NORTH_LEFT)
EAST_STRAIGHT = (EAST_RIGHT, WEST_LEFT)
NORTH_STRAIGHT = (NORTH_RIGHT, SOUTH_LEFT)

WEST_RIGHT_TURN = turn_road(WEST_RIGHT, SOUTH_LEFT, TURN_RIGHT, n)
WEST_LEFT_TURN = turn_road(WEST_RIGHT, NORTH_LEFT, TURN_LEFT, n)

SOUTH_RIGHT_TURN = turn_road(SOUTH_RIGHT, EAST_LEFT, TURN_RIGHT, n)
SOUTH_LEFT_TURN = turn_road(SOUTH_RIGHT, WEST_LEFT, TURN_LEFT, n)

EAST_RIGHT_TURN = turn_road(EAST_RIGHT, NORTH_LEFT, TURN_RIGHT, n)
EAST_LEFT_TURN = turn_road(EAST_RIGHT, SOUTH_LEFT, TURN_LEFT, n)

NORTH_RIGHT_TURN = turn_road(NORTH_RIGHT, WEST_LEFT, TURN_RIGHT, n)
NORTH_LEFT_TURN = turn_road(NORTH_RIGHT, EAST_LEFT, TURN_LEFT, n)

sim3.create_roads([
    WEST_INBOUND,
    SOUTH_INBOUND,
    EAST_INBOUND,
    NORTH_INBOUND,

    WEST_OUTBOUND,
    SOUTH_OUTBOUND,
    EAST_OUTBOUND,
    NORTH_OUTBOUND,

    WEST_STRAIGHT,
    SOUTH_STRAIGHT,
    EAST_STRAIGHT,
    NORTH_STRAIGHT,

    *WEST_RIGHT_TURN,  # so this is n little roads
    *WEST_LEFT_TURN,   # and this is too

    *SOUTH_RIGHT_TURN,
    *SOUTH_LEFT_TURN,

    *EAST_RIGHT_TURN,
    *EAST_LEFT_TURN,

    *NORTH_RIGHT_TURN,
    *NORTH_LEFT_TURN
])

def road(a):
    # build many (n, from above, to be precise) small roads to model a turn
    return range(a, a+n)    

sim3.create_gen({
'vehicle_rate': 40,   # changed this from previous sim
'vehicles':[
    [4, {'path': [0, 8, 6], 'direction': 'STRAIGHT'}],   # [WEST_INBOUND, WEST_STRAIGHT, EAST_OUTBOUND],  from above roads array
    [1, {'path': [0, *road(12), 5], 'direction': 'RIGHT'}],   # [WEST_INBOUND, *WEST_RIGHT_TURN, SOUTH_OUTBOUND],  from above roads array
    [1, {'path': [0, *road(12+n), 7], 'direction': 'LEFT'}],   # [WEST_INBOUND, *WEST_LEFT_TURN, NORTH_OUTBOUND],  from above roads array

    [2, {'path': [1, 9, 7], 'direction': 'STRAIGHT'}],   # [SOUTH_INBOUND, SOUTH_STRAIGHT, NORTH_OUTBOUND]
    [1, {'path': [1, *road(12+2*n), 6], 'direction': 'RIGHT'}],   # etc
    [2, {'path': [1, *road(12+3*n), 4], 'direction': 'LEFT'}],


    [1, {'path': [2, 10, 4], 'direction': 'STRAIGHT'}],  # [EAST_INBOUND, EAST_STRAIGHT, EAST_OUTBOUND]
    [1, {'path': [2, *road(12+4*n), 7], 'direction': 'RIGHT'}],  # etc
    [1, {'path': [2, *road(12+5*n), 5], 'direction': 'LEFT'}],

    [1, {'path': [3, 11, 5], 'direction': 'STRAIGHT'}],  # [NORTH_INBOUND, etc]
    [2, {'path': [3, *road(12+6*n), 4], 'direction': 'RIGHT'}],
    [4, {'path': [3, *road(12+7*n), 6], 'direction': 'LEFT'}]
]})

sim3.create_signal([[0],[1],[2],[3]]) ;  # using 4 separate groups here so that each stop sign can function individually
                                        # Though my signals are permanently on "red light" for my project

Note that if you run the cell below, it should open a pygame window that shows the simulation, which may or may not be what you want.  But if so, you can stop it by coming back to this notebook and hitting the black square under the word "Kernel" in the toolbar at the top.

In [5]:
# Start simulation

win = Window(sim3)
win.zoom = 5  # use two fingers to change this zoom while running
win.run(steps_per_update=3)  # controls the speed of the visualization (higher = faster)

KeyboardInterrupt: 

In [6]:
SIM_3_DF = logDF(sim3.popped)
SIM_3_DF.head()

,Road,enter_Q,time_needed,exit_Q,direction
0,2,14.48,2,15.45,RIGHT
1,1,15.99,2,17.61,RIGHT
2,1,19.12,2,20.22,STRAIGHT
3,1,21.69,2,22.80,STRAIGHT
4,0,22.03,3,23.96,LEFT


In [9]:
len(sim3.popped)

118

That one got backed up coming from the West towards the popular East direction, which maybe makes sense.  The weightings for that West-East road were 4 straight to 1 left and 1 right.  But the South-North road eventually jammed up too, so I cut it off at 118 cars through.  There's a precarious balance with the physics and the threading colliding here like the cars in the intersections, and each configuration really needs its own dynamics to work, the way I have it designed.  For the sake of a good flow into SimPy, I'll use the data from SIM_1 for now, and hope to come back to this project in the future with some clever adjustments.

In [10]:
#SIM_3_DF.to_csv("SIM_3_DF.csv")

## Part 2 -- Feeding the cars into SimPy to see how efficient it is with different traffic rules.

I'll feed the cars into the simpy "intersection" one by one first, as they entered the simulated flow intersection above.   Then I'll see how much quicker things flow if cars are allowed to alternate turns N-S roads then E-W roads, where there are pairs of such cars queued up and one is going straight while the other is not turning left.  So just like real life.

In [51]:
env = sp.Environment()
turn = sp.PriorityResource(env, capacity=1)  # one car at a time thru intersection, for now
log = []

def next_car(log, envir, resource, road, direction, t_req, t_queue, priority=1, name='carl'):
    
    times = {"LEFT": 3, "STRAIGHT": 2, "RIGHT": 2}
    
    with resource.request(priority=priority) as req:
        #print(f'{name} on road {road} requesting at {t_enter} with priority={priority}')
        yield req
        #print(f'{name} on road {road} entered the intersection at {t_enter}')
        yield env.timeout(times[direction] + t_queue)
        #print(f'{name} on road {road} left intersection at {env.now}')
        log.append((env.now, t_enter, road, direction, name))
    return log

These times through the intersection (left turn = 3 units, right and straight = 2 each) are the same as the ones I used for the generator simulator.  There's a big difference between the two uses, of course.  The times aren't to compare SimPy to pygame, but just to compare SimPy to SimPy.  The pygame numbers involve all the nastiness of a busy intersection, and if you watched the simulations above, you noticed how cars were colliding often in the intersections, and cutting it pretty close many other times. But if there was strict alternation, better coding, and perfect drivers, those are pretty much the times it would've taken the cars to enter and exit the intersection.  

All those cars were individual Agents in Part 1, subject to all sorts of fluctuations, and it's actually fun to stare at it for awhile and realize that "someone is just sitting there jamming things up, even though it's their turn."  You can almost hear the honking and see the birds being flipped at the person who is texting at the wheel.  But SimPy will offer a nice, clean break from that real-world nonsense, by reducing all the Agents and threads and physics to just one omnipotent machine of reason.  

In [13]:
flowDF = pd.read_csv('SIM_1_DF.csv', index_col=0)
flowDF.head()

,Road,enter_Q,time_needed,exit_Q,direction
0,2,14.48,2,15.45,STRAIGHT
1,1,15.99,2,17.61,RIGHT
2,1,19.12,2,20.22,STRAIGHT
3,1,21.69,2,22.80,STRAIGHT
4,0,23.54,2,25.03,STRAIGHT


In [14]:
flowDF.tail()

,Road,enter_Q,time_needed,exit_Q,direction
245,3,387.92,2,390.29,STRAIGHT
246,2,388.42,3,390.35,LEFT
247,3,391.51,2,393.63,RIGHT
248,1,391.64,2,393.64,RIGHT
249,2,391.70,2,393.69,STRAIGHT


Looking at the head and tail of the data there, you can see, as expected, that the time spent in the intersection queue got longer as time went on and traffic backed up somewhat.  The first five cars through spent about 1-1.5 seconds each from they time they turned yellow in the simulation to when they turned green again as they entered the intersection.  The last five times were about 2 seconds each, and that's the simulation where things were flowing most smoothly.  

It's debatable whether to use the entry time or the exit time, or maybe the average of the two.  But I'll use the entry time here to feed into SimPy, and to compensate, I'll add the waiting time into the clean, theoretical times.  Again, the point isn't to compare simpy to the previous flow simulation that generated the data, but the point is to model reality, and the reality is that things take longer as traffic builds.  

In [16]:
flowDF['t_q'] = flowDF.exit_Q - flowDF.enter_Q
flowDF.head(1)

,Road,enter_Q,time_needed,exit_Q,direction,t_q
0,2,14.48,2,15.45,STRAIGHT,0.97


In [41]:
for i in flowDF.index:
    car = flowDF.loc[i]
    env.process(next_car(log, env, turn, car.Road, car.direction,
                         car.t_q, priority=1, name='car_' + str(i)))

In [42]:
env.run()

In [45]:
pd.DataFrame(log, columns=["out", "in", 'road', 'direction', 'name'])

,out,in,road,direction,name
0,2.97,0,2,STRAIGHT,car_0
1,6.59,0,1,RIGHT,car_1
2,9.69,0,1,STRAIGHT,car_2
3,12.80,0,1,STRAIGHT,car_3
4,16.29,0,0,STRAIGHT,car_4
...,...,...,...,...,...
245,1083.53,0,3,STRAIGHT,car_245
246,1088.46,0,2,LEFT,car_246
247,1092.58,0,3,RIGHT,car_247
248,1096.58,0,1,RIGHT,car_248


In [54]:
from simpy.util import start_delayed
env = sp.Environment()
def sub(env):
    print(env.now)
    yield env.timeout(1)
    
    return 23
def parent(env):
    sub_proc = yield start_delayed(env, sub(env), delay=3)
    ret = yield sub_proc
    return ret
env.run(env.process(parent(env)))


3


23

In [50]:
start_delayed?

In [51]:
env = sp.Environment()
turn = sp.PriorityResource(env, capacity=1)  # one car at a time thru intersection, for now
log = []

def next_car(log, envir, resource, road, direction, t_req, t_queue, priority=1, name='carl'):
    
    times = {"LEFT": 3, "STRAIGHT": 2, "RIGHT": 2}
    
    with resource.request(priority=priority) as req:
        #print(f'{name} on road {road} requesting at {t_enter} with priority={priority}')
        yield req
        #print(f'{name} on road {road} entered the intersection at {t_enter}')
        yield env.timeout(times[direction] + t_queue)
        #print(f'{name} on road {road} left intersection at {env.now}')
        log.append((env.now, t_enter, road, direction, name))
    return log